In [1]:
import gc
import pandas as pd
import numpy as np
import pyarrow as pa
import tensorflow as tf
from pyarrow import parquet as pq
from collections import defaultdict

In [2]:
train = pd.read_csv('../Data/Train/sales_train_validation.csv')
train = train.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis =1)
train.index = train.id
train = train.drop('id',axis = 1)

In [7]:
def _rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def _get_time_tensor(arr,window_size):
    tmp = _rolling_window(arr,window_size+1)
    Xtensor = tmp[:,:-1]
    Ytensor = tmp[:,-1]
    return (Xtensor.reshape(-1,window_size,1),Ytensor.reshape(-1,1))

def np_to_time_tensor_generator(arr,windowSize):
    if np.ndim(arr) > 1:
        for ix,v in enumerate(arr):
            yield _get_time_tensor(v,windowSize)
    else:
        yield _get_time_tensor(arr,windowSize) 

def _get_item_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[:3])

def _get_store_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[3:5])

def _get_cate_info(fullIndex,cateInfoDir):
    item_id = _get_item_id(fullIndex)
    store_id = _get_store_id(fullIndex)
    return pd.read_parquet(cateInfoDir,filters = [("item_id",'=',str(item_id)),("store_id",'=',str(store_id))])

def _get_events(calendar_dir,sdate,edate):
    df = pd.read_csv(calendar_dir)
    df['d_num'] = df.d.apply(lambda x: x.replace('d_','')).astype('int')
    return df[df.d_num.apply(lambda x: x <= edate and x >= sdate)]

def _get_future_events(calendar_dir, sdate,duration):
    edate = sdate+duration-1
    return _get_events(calendar_dir,sdate,edate)


In [30]:
train_univariate = tf.data.Dataset.from_generator(
    np_to_time_tensor_generator,
    (tf.float32,tf.float32),
    output_shapes = (tf.TensorShape([None,100,1]),tf.TensorShape([None,1])),
    args = (train.head(100).values,100))
train_univariate = train_univariate.prefetch(5).repeat()

In [31]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=(100,1)),
    tf.keras.layers.Dense(1)
])
simple_lstm_model.compile(optimizer='adam', loss='mae')

In [32]:
EVALUATION_INTERVAL = 2
EPOCHS = 10

simple_lstm_model.fit(
    train_univariate,
    epochs=EPOCHS,
    steps_per_epoch= EVALUATION_INTERVAL,
    validation_data=train_univariate, 
    validation_steps=50
    )

Train for 2 steps, validate for 50 steps
Epoch 1/10
2/2 [==============================] - 8s 4s/step - loss: 0.3130 - val_loss: 1.3334
Epoch 2/10
2/2 [==============================] - 1s 633ms/step - loss: 0.9513 - val_loss: 1.3333
Epoch 3/10
2/2 [==============================] - 1s 578ms/step - loss: 0.9545 - val_loss: 1.3319
Epoch 4/10
2/2 [==============================] - 1s 565ms/step - loss: 3.8233 - val_loss: 1.3307
Epoch 5/10
2/2 [==============================] - 1s 607ms/step - loss: 0.9451 - val_loss: 1.3299
Epoch 6/10
2/2 [==============================] - 1s 637ms/step - loss: 0.2469 - val_loss: 1.3289
Epoch 7/10
2/2 [==============================] - 1s 531ms/step - loss: 0.7894 - val_loss: 1.3282
Epoch 8/10
2/2 [==============================] - 1s 568ms/step - loss: 5.6839 - val_loss: 1.3270
Epoch 9/10
2/2 [==============================] - 1s 635ms/step - loss: 0.5707 - val_loss: 1.3257
Epoch 10/10
2/2 [==============================] - 1s 547ms/step - loss: 2.6379 

In [ ]:
# tensorflow